In [ ]:
!pip install ktrain

     |████████████████████████████████| 25.2MB 1.3MB/s 
     |████████████████████████████████| 421.8MB 37kB/s 
     |████████████████████████████████| 6.7MB 50.5MB/s 
     |████████████████████████████████| 983kB 24.6MB/s 
     |████████████████████████████████| 245kB 51.4MB/s 
     |████████████████████████████████| 675kB 47.6MB/s 
     |████████████████████████████████| 471kB 51.9MB/s 
     |████████████████████████████████| 3.9MB 47.5MB/s 
     |████████████████████████████████| 450kB 48.0MB/s 
     |████████████████████████████████| 1.1MB 45.8MB/s 
     |████████████████████████████████| 890kB 47.3MB/s 
     |████████████████████████████████| 3.8MB 48.1MB/s 
  Created wheel for ktrain: filename=ktrain-0.16.3-cp36-none-any.whl size=25246180 sha256=a4a3a2459c3f9182d42f4604dbf9faa8c188642541fc4a2d828952988895b3c8
  Stored in directory: /root/.cache/pip/wheels/2b/24/21/a52cc4dcb0b438cf570e9d44c017761d41335d9af4c189ebad
  Created wheel for keras-bert: filename=keras_bert-0.84.0-cp36-no

In [ ]:
import ktrain 
from ktrain import text

import pandas as pd
import numpy as np
import re
import os
import tensorflow as tf

In [ ]:
!gdown --id 1S6qMioqPJjyBLpLVz4gmRTnJHnjitnuV
!gdown --id 1zdmewp7ayS4js4VtrJEHzAheSW-5NBZv

Downloading...
From: https://drive.google.com/uc?id=1S6qMioqPJjyBLpLVz4gmRTnJHnjitnuV
To: /content/apps.csv
100% 134k/134k [00:00<00:00, 62.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1zdmewp7ayS4js4VtrJEHzAheSW-5NBZv
To: /content/reviews.csv
7.17MB [00:00, 113MB/s]


In [ ]:
data = pd.read_csv('reviews.csv')

In [ ]:
data.head(2)

,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,sortOrder,appId
0,Andrew Thomas,https://lh3.googleusercontent.com/a-/AOh14GiHd...,Update: After getting a response from the deve...,1,21,4.17.0.3,2020-04-05 22:25:57,"According to our TOS, and the term you have ag...",2020-04-05 15:10:24,most_relevant,com.anydo
1,Craig Haines,https://lh3.googleusercontent.com/-hoe0kwSJgPQ...,Used it for a fair amount of time without any ...,1,11,4.17.0.3,2020-04-04 13:40:01,It sounds like you logged in with a different ...,2020-04-05 15:11:35,most_relevant,com.anydo


In [ ]:
df = data.filter(['content', 'score'], axis=1)

In [ ]:
df['score'].unique()

array([1, 2, 3, 4, 5])

In [ ]:
def clean_txt(text):
  text = re.sub('[^a-zA-Z]', ' ', text)
  text = text.lower()
  text = text.split()
  text = [w for w in text if w not in stopword.stopwords('english')]
  text = ' '.join(text)
  return text

In [ ]:
def score_change(score):
  if score <=2:
    return 'neg'
  elif score ==3:
    return 'neutral'
  else:
    return 'pos'

In [ ]:
df['score'] = df.score.apply(score_change)

In [ ]:
df['score'].unique()

array(['neg', 'neutral', 'pos'], dtype=object)

In [ ]:
df.score.unique()

array(['neg', 'neutral', 'pos'], dtype=object)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train, test = train_test_split(df, test_size = 0.2, random_state = 42)

In [ ]:
x_train = np.array(train.content)
y_train = np.array(train.score)

x_test = np.array(test.content)
y_test = np.array(test.score)

In [ ]:
type(x_train)

numpy.ndarray

In [ ]:
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=x_train, y_train=y_train,
                                                                      x_test = x_test, y_test=y_test, 
                                                                      class_names=['neg', 'neutral', 'pos'],
                                                                      preprocess_mode='bert', maxlen=150)

task: text classification
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


/usr/local/lib/python3.6/dist-packages/ktrain/text/preprocessor.py:519: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  if self.get_classes(): warnings.warn('class_names argument was ignored, as they were extracted from string labels in dataset')


In [ ]:
model = text.text_classifier(name='bert', train_data=(x_train, y_train), preproc=preproc)

learner = ktrain.get_learner(model, train_data=(x_train, y_train), val_data=(x_test, y_test), batch_size=6)

Is Multi-Label? False
maxlen is 150
done.


In [ ]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)


In [ ]:
learner.fit_onecycle(lr=2e-5, epochs=1, callbacks = [cp_callback])



begin training using onecycle policy with max lr of 2e-05...
Train on 12596 samples, validate on 3150 samples
12594/12596 [============================>.] - ETA: 0s - loss: 0.7421 - accuracy: 0.6517
Epoch 00001: saving model to training_1/cp.ckpt
12596/12596 [==============================] - 972s 77ms/sample - loss: 0.7421 - accuracy: 0.6518 - val_loss: 0.5993 - val_accuracy: 0.7362


In [ ]:
!ls {checkpoint_dir}

checkpoint		     cp.ckpt.data-00001-of-00002
cp.ckpt.data-00000-of-00002  cp.ckpt.index


In [ ]:
model.load_weights(checkpoint_path)


In [ ]:

text = ' racism shit this is. Rape and sex'

In [ ]:
txt = 'I am a Good Boy'

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [ ]:
predictor.predict(text)

'neg'

In [ ]:
predictor.predict(txt)

'pos'

***QNA BERT***

In [ ]:
Messi = '/content/drive/My Drive/Messi'

In [ ]:
INDEXDIR = '/temporary/myindex'

text.SimpleQA.initialize_index(INDEXDIR)
text.SimpleQA.index_from_folder(folder_path=Messi, index_dir=INDEXDIR)

In [ ]:
qna = text.SimpleQA(INDEXDIR)

In [ ]:
answer = qna.ask("When was Lionel Messi born ?")
ans = answer[0]['answer']
print(ans)

on 24 june 1987


In [ ]:
answer = qna.ask("How many goals Messi scored in 2008-09 season ?")
ans = answer[0]['answer']
print(ans)

scored 38


In [ ]:
answer = qna.ask("When Lionel Messi became the youngest argentine to play in tha FIFA world cup ?")
ans = answer[0]['answer']
print(ans)

in 2006


In [ ]:
answer = qna.ask("Which diagonsis Messi had in his childhood ?")
ans = answer[0]['answer']
print(ans)

a growth hormone deficiency


In [ ]:
answer = qna.ask("When Messi won his first ballon d'or ?")
ans = answer[0]['answer']
print(ans)

of 22


In [ ]:
answer = qna.ask("When Messi made hid debut in Laliga ?")
ans = answer[0]['answer']
print(ans)

the 2004 – 05 season


In [ ]:
answer = qna.ask("Whose successor was Lionel Messi ?")
ans = answer[0]['answer']
print(ans)

diego maradona


***TEXT SUMMARAZATION WITH BERT***






In [ ]:
ts = text.TransformerSummarizer()

In [ ]:
sum_messi = ts.summarize(Messi)

In [ ]:
messi = sum_messi.split('.')

for s in messi:
  print(f'{s}.', '\n')

Messi is the current world No. 

 1. 

 He has scored more goals than any other player in the history of the game. 

 He also has the most assists in the Premier League this season. 

 He is the first player to score more goals in the Champions League than any player in history. 

. 



In [ ]:
love = ts.summarize("Love encompasses a range of strong and positive emotional and mental states, from the most sublime virtue or good habit, the deepest interpersonal affection and to the simplest pleasure. An example of this range of meanings is that the love of a mother differs from the love of a spouse, which differs from the love of food. Most commonly, love refers to a feeling of strong attraction and emotional attachment. Love is considered to be a positive and negative: with its virtue representing human kindness, compassion, and affection, as 'the unselfish loyal and benevolent concern for the good of another'; and its vice representing human moral flaw, akin to vanity, selfishness, amour-propre, and egotism, as it potentially leads people into a type of mania, obsessiveness or codependency. It may also describe compassionate and affectionate actions towards other humans, one's self or animals. In its various forms, love acts as a major facilitator of interpersonal relationships and, owing to its central psychological importance, is one of the most common themes in the creative arts. Love has been postulated to be a function to keep human beings together against menaces and to facilitate the continuation of the species.")


In [ ]:
love =love.split('.')

for s in love:
  print(f'{s}.', '\n')

Love is considered to be a positive and negative: with its virtue representing human kindness, compassion, and affection. 

 It may also describe compassionate and affectionate actions towards other humans, one's self or animals. 

 Love acts as a major facilitator of interpersonal relationships and, owing to its central psychological importance, is one of the most common themes in the creative arts. 

. 

